---
title: "2023_Kronensicherung_Plesse_Analyse"
author: "Kyell Jensen"
date: "2024-08-06"
format: pdf
editor: visual
---

# 2023_Kronensicherung_Plesse_Analyse

## Kombinierte Analyse LineScale3, TreeQinetic und Versuchsaufzeichung

Nutze eine geeignete Python 3.11 Umgebung (z. B. virtuelle Environment).

## Arbeitsumgebung vorbereiten

Nutze gemeinsame imports.py für alle Analysen-Notebooks.
Erstelle Instanzen für Config und PlotManager.
Lade Dateinamen.

In [3]:
from project_setup import load_config_and_plot_manager, load_data
from project_config import data_export_directory, filename_clean_dataset, filename_clean_data_dict


In [4]:
# Lade Konfiguration und PlotManager
CONFIG, PLOT_MANAGER, logger = load_config_and_plot_manager()

# Lade Datensatz und Data Dictionary
df, data_dict = load_data(logger, data_export_directory, filename_clean_dataset, filename_clean_data_dict)

2025-04-06 12:19:48 [WARNING] kj_core.core_config.set_working_directory: The directory C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\combined already exists.
2025-04-06 12:19:48 [INFO] kj_core.core_config.set_working_directory: Working directory set to C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\combined!
2025-04-06 12:19:48 [INFO] kj_core.core_config.__init__: <CoreConfig>
  Package:            kj_core
  Working Directory:  C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\combined
  Plot Directory:     C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\combined\plots
  Data Directory:     C:\kyellsen\005_Projekte\2024

## IMPORT: Daten Import

Lege Pfade für Daten-Importe, Daten-Exporte etc. fest (ggf. anpassen an eigene Verzeichnisstruktur), ausgelagert in gemeinsame Config für verschiedene Notebooks

In [5]:
filename_dataset = "_dataset_clean.feather"
filename_data_dict = "_data_dict_clean.json"

In [6]:
# Dateien laden
df = pd.read_feather(data_export_directory / filename_dataset)

with open(data_export_directory / filename_data_dict, "r", encoding="utf-8") as f:
    data_dict = json.load(f)

NameError: name 'pd' is not defined

## ANALYSE: Explorative Datenanalyse

In [ ]:
# In DataFrame umwandeln
data_dict_df= build_data_dict_df(data_dict)

# In Markdown umwandeln und anzeigen
md_text = data_dict_df.to_markdown(tablefmt="github")
display(Markdown(md_text))

In [ ]:
df.head(10)

In [ ]:
df.columns

### COMBINED: Definition von Darstellungsstandards
Festlegen von Farbcodes für einheitliche Darstellung von Sensoren und Behandlungsvarianten für alle nachfolgenden Plots.

In [ ]:
color_palette = PLOT_MANAGER.color_palette_list

# Für die Spalte "treatment":
treatment_color_dict = get_color_dict(df, "treatment", PLOT_MANAGER.color_palette_list)
# Für die Spalte "sensor_name":
sensor_color_dict = get_color_dict(df, "sensor_name", PLOT_MANAGER.color_palette_list)

### Spitzenlasten in der KS, maximale Messung Beispieldaten

In [ ]:
# Filtern der Einträge mit dem maximalen Wert in 'cable_max'
max_cable_max = df['cable_max'].max()
filtered_df = df[df['cable_max'] == max_cable_max]

# Innerhalb der gefilterten Einträge den maximalen 'max_strain' finden
max_value_row = filtered_df.loc[filtered_df['max_strain'].idxmax()]
max_value_row

In [ ]:
# Liste der gewünschten Spalten in der gewünschten Reihenfolge
columns_to_display = [
    'id', 'treatment', 'release_force_target', 'rope_release', 
    'cable_max', 'sensor_name', 'location', 'height', 'diameter', 'max_strain', 'max_compression'
]

# Zeile auf die gewünschten Spalten in der angegebenen Reihenfolge beschränken
max_value_row_filtered = max_value_row[columns_to_display]

max_value_row_filtered['treatment'] = slugify(max_value_row_filtered['treatment'])

# Erstelle ein DataFrame mit den zusätzlichen Informationen aus `data_dict`
expanded_data = []
for col in columns_to_display:
    expanded_data.append({
        "Zeichen": data_dict[col]["Zeichen"],
        "Deutsch": data_dict[col]["Deutsch"],
        "Wert": max_value_row_filtered[col],
        "Einheit": data_dict[col]["Einheit"],
    })

# Neues DataFrame erstellen
expanded_df = pd.DataFrame(expanded_data)
expanded_df

In [ ]:
# LaTeX-String erstellen
latex_string = expanded_df.to_latex(
    index=False, 
    escape=False, 
    column_format="llrr",  # Spaltenformat angepasst
    float_format="{:0.2f}".format
)

caption = "Feldversuch 2 - Ergebnisse, Werte der Messung mit Spitzenlast"
caption_long = "Feldversuch 2 - Ergebnisse, Messung mit der höchsten Kraftspitze in der KS (nur Werte des Elastometers mit max. Faserdehnung)"

save_latex_table(latex_string, caption, latex_export_directory, caption_long)